https://discourse.pymc.io/t/a-bayesian-approach-to-media-mix-modeling-by-michael-johns-zhenyu-wang/6024

http://www.guillaumenicaise.com/wp-content/uploads/2013/10/Borden-1984_The-concept-of-marketing-mix.pdf

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

predictors = weekly marketing spend in each channel, transformed
* non linear functional transformation to account for diminishing returns on spend
* adstock tranformation of channel spend to account for lagged effects of advertising

Reach function: non linear shape function with a single parameter mu to capture diminishing returns on spend
<br>mu controls how quickly the curve will saturate. mu = 1 gives a linear curve
$$
[\frac{1 - e^{-\mu x_mt}}{1 + e^{-\mu x_mt}}]
$$


Adstock to capture carry over effects of marketing: allow marketing signal to decay over time<br>
This is a geometric decay. <br>
As alpha goes up, the decay rate slows down and the impact of the marketing is assumed over a longer period of time

$$
[\frac{\sum_{l=0}^{t-1} \alpha^l x_{t-l}}{\sum_{l=0}^{t-1}\alpha^l}\
$$

Bayesian methods can
1. set constraints on model parameters
2. incorporate existing knowledge using priors
3. help learn saturation and decay parameters. as a part of model fitting
4. produce a generative model that is well suited simulation

Base model likelihood and prior structure
$$
\begin{equation}
y_t = \alpha + \sum_{m=1}^{M} \beta_m f(x_{t,m}) + \sum_{c=1}^c \beta_cZ_{t,c} + e_t
\end{equation}
$$
where

$y_t$ = new customers expected value (scaled) \
$\beta_m$ = is the marketing channel that is instantiated with a prior, m is the number of marketing channels \
$\beta_c$ = the beta for the control variable  \
$\alpha$ = a prior for the alpha parameter for the adstock transformation we apply to a subset for our channels. \
$mu$ = A prior for mu that controls the saturation curve for each channel. \
$e_t$ = prior for our noise term\
m = number of marketing channels

#### Base model priors
The likelihood is a normal distribution
mu is a function of the linear model and sigma will stand in for the standard deviation
This has 5 control variables and 12 channels. All marketing channels are transformed with the saturation function but only a subset is transformed with the adstock function to represent and try to capture decay and effects. All variables are scaled between [0,1]

1. The beta priors are half normal so spends are never negative. 
2. alpha channels/adstock function is a beta distribution https://docs.pymc.io/api/distributions/continuous.html#pymc3.distributions.continuous.Beta
3. For saturation, the prior is a gamma https://docs.pymc.io/api/distributions/continuous.html#pymc3.distributions.continuous.Gamma
4. sigma term is an exponential distribution

beta_Control_l = Normal(mu = 0.0, sigma = 0.25) <br> 
beta_Control_2 = Normal(mu = 0.0, sigma = 0.25) <br> 
beta_Control_3 = Normal(mu = 0.0, sigma = 0.25) <br> 
beta_month_ = Normal(mu = 0.0, sigma = 0.25) <br> 
beta_year_2020 Normal(mu = 0.0, sigma = 0.25) <br> 
beta_Channel_1_spend_scale = HalfNormal(sigma = 5.0) <br> 
alpha_Channel_1_spend_scale = Beta(alpha = 3.0, beta = 3.0) <br> 
mu_Channel_l_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel_10_spend_scale = HalfNormal(sigma = 5.0) <br>
alpha_Channel_10_spend_scale = Beta(alpha = 3.0. beta = 3.0) <br> 
mu_Channel_10_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel_2_spend_scale = HalfNormal(sigma = 5.0) <br> 
alpha_Channel_2_spend_scale = Beta(alpha = 3.0 beta = 3.0) <br> 
mu_Channel_2_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel_3_spend_scale = HalfNormal(sigma = 5.0) <br> 
alpha_Channel_3_spend_scale = Beta(alpha = 3.0. beta = 3.0) <br> 
mu_Channel3_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel4_spend_scale = HalfNormal(sigma = 5.0) <br> 
muChannel_4_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel_7_spend_scale = HalfNormal(sigma = 5.0) <br> 
mu_Channel_7_spend_scale = Gamma(alpha = 3.0, beta = 1.0) <br> 

beta_Channel_5_spend_scale = HalfNormal(sigma = 5.0) <br> 
muChannel_5_spend_scale = Gamma(alpha = 3.0. beta = 1.0) <br> 

beta_Channel_6_spend_scale = HalfNormal(sigma = 5.0) <br> 
mu_Channel_6_spend_scale = Gamma(alpha = 3.0, beta = 1.0) <br> 

beta_Channel_9_spend_scale = HalfNormal(sigma = 5.0) <br> 
mu_Channel_9_spend_scale = Gamma(alpha = 3.0, beta = 1.0) <br> 

beta_Channel_11_spend_scale = HalfNormal(sigma = 5.0) <br> 
mu_Channel_ll_spend_scale Gamma(alpha = 3.0, beta = 1.0) <br> 

beta_Channel_12_spend_scale = HalfNormal(sigma = 5.0) <br> 
mu_Channel_12_spend_scale Gamma(alpha = 3.0. been = 1.0) <br> 

sigma = Exponential(lam = 10.0) 


### Data sample
12 weeks of user registrations with 12 marketing channels

In [22]:
import numpy as np
import pandas as pd
import pymc3 as pm
from pymc3 import *
import arviz as az

In [3]:
d = {}
for i in range(1, 13, 1):
    d[f"channel_{i}"] = np.random.uniform(0, 1, 12)

In [4]:
df_in = pd.DataFrame(d)

In [5]:
df_in["y"] = np.random.uniform(0, 1, 12)

### Model layout

In [6]:
delay_channels = ["channel_1", "channel_2", "channel_3", "channel_10"] # channels that can have both decay and saturation effects
non_lin_channels = ["channel_4", "channel_5", "channel_6", "channel_7", 
                    "channel_12", "channel_11", "channel_9", "channel_8"]

In [7]:
def logistic_function(x_t, mu=0.1):
    return (1 - np.exp(-mu * x_t)) / (1 + np.exp(-mu * x_t))

import theano.tensor as tt
def geometric_adstock_tt(x_t, alpha=0, L=12, normalize=True):
    w = tt.as_tensor_variable([tt.power(alpha, i) for i in range(L)])
    xx = tt.stack([tt.concatenate([tt.zeros(i), x_t[:x_t.shape[0] - i]]) for i in range(L)])
    
    if not normalize:
        y = tt.dot(w, xx)
    else:
        y = tt.dot(w // tt.sum(w), xx)
    return y

In [8]:
with Model() as model:
    response_mean = []
    # channels that can have decay and saturation effects.
    for channel_name in delay_channels:
        xx = df_in[channel_name].values
        print(f"Adding delayed channel: {channel_name}")
        channel_b = HalfNormal(f"beta_{channel_name}", sd=5)
        alpha = Beta(f"alpha_{channel_name}", alpha=3, beta=3)
        channel_mu = Gamma(f"mu_{channel_name}", alpha=3, beta=1)
        # we transform the marketing spend with the adstock and then that transformed version is fed to the logistic
        # reach function that models our saturation. 
        response_mean.append(logistic_function(geometric_adstock_tt(xx, alpha), channel_mu) * channel_b)
    
    # channels that can have decay and saturation effects
    for channel_name in non_lin_channels:
        xx = df_in[channel_name].values
        
        print(f"Adding non linear logistic channel: {channel_name}")
        channel_b = HalfNormal(f"beta_{channel_name}", sd=5)
        
        # logistic reach curve
        channel_mu = Gamma(f"mu_{channel_name}", alpha=3, beta=1)
        response_mean.append(logistic_function(xx, channel_mu) * channel_b)
        
    # Continuous control variables
#     if control_vars:
#         for channel_name in control_vars:
#             x = df_in[channel_name]
#             print(f"adding control: {channel_name}")
#             control_beta = Normal(f"beta_{channel_name}", s=0.25)
#             channel_contrib = control_beta * x
#             response_mean.append(channel_contrib)
    
#     # Categorical control variables.
#     if index_vars:
#         for var_name in index_vars:
#             shape = len(df_in[var_name].unique())
#             x = df_in[var_name].values
            
#             print(f"adding index variable : {var_name}")
#             ind_beta = Normal(f"beta_{var_name}", sd=5, shape=shape)
#             channel_contrib = ind_beta[x]
#             response_mean.append(channel_contrib)
            
    sigma = Exponential("sigma", 10)
    likelihood = Normal("y", mu=sum(response_mean), sd=sigma, observed=df_in["y"].values)

Adding delayed channel: channel_1
Adding delayed channel: channel_2
Adding delayed channel: channel_3
Adding delayed channel: channel_10
Adding non linear logistic channel: channel_4
Adding non linear logistic channel: channel_5
Adding non linear logistic channel: channel_6
Adding non linear logistic channel: channel_7
Adding non linear logistic channel: channel_12
Adding non linear logistic channel: channel_11
Adding non linear logistic channel: channel_9
Adding non linear logistic channel: channel_8


In [20]:
with model:
     trace = pm.sample(10000, tune=1000, chains=2, init="adapt_diag", random_seed=666, return_inferencedata=True, cores=1)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, mu_channel_8, beta_channel_8, mu_channel_9, beta_channel_9, mu_channel_11, beta_channel_11, mu_channel_12, beta_channel_12, mu_channel_7, beta_channel_7, mu_channel_6, beta_channel_6, mu_channel_5, beta_channel_5, mu_channel_4, beta_channel_4, mu_channel_10, alpha_channel_10, beta_channel_10, mu_channel_3, alpha_channel_3, beta_channel_3, mu_channel_2, alpha_channel_2, beta_channel_2, mu_channel_1, alpha_channel_1, beta_channel_1]


Sampling 2 chains for 1_000 tune and 10_000 draw iterations (2_000 + 20_000 draws total) took 135 seconds.
There were 108 divergences after tuning. Increase `target_accept` or reparameterize.
There were 444 divergences after tuning. Increase `target_accept` or reparameterize.


In [23]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_channel_1,3.995,3.021,0.001,9.294,0.026,0.018,8806.0,5442.0,1.0
alpha_channel_1,0.501,0.190,0.157,0.846,0.002,0.001,14516.0,10604.0,1.0
mu_channel_1,2.999,1.745,0.327,6.162,0.015,0.011,10428.0,8417.0,1.0
beta_channel_2,4.004,2.980,0.000,9.367,0.028,0.020,7357.0,4459.0,1.0
alpha_channel_2,0.501,0.190,0.155,0.846,0.002,0.001,15039.0,10091.0,1.0
mu_channel_2,2.986,1.717,0.297,6.107,0.015,0.010,11512.0,7730.0,1.0
beta_channel_3,3.962,3.021,0.000,9.328,0.027,0.019,8376.0,5355.0,1.0
alpha_channel_3,0.499,0.188,0.156,0.841,0.002,0.001,12878.0,9537.0,1.0
mu_channel_3,3.000,1.703,0.324,6.033,0.016,0.011,9736.0,8392.0,1.0
beta_channel_10,4.014,2.973,0.002,9.373,0.027,0.019,8043.0,5148.0,1.0
